In [1]:
#Imports das classes criadas
from Classes.Arquivo import Arquivo
from Classes.BancoDados import BancoDados
from Classes.BancoDDL import BancoDDL
from Classes.BancoDML import BancoDML
from Classes.BancoDQL import BancoDQL
import plotly.express as px #py -3.12 -m pip install plotly
import pandas as pd

In [3]:
#Carregando todos os CSVs e os inspecionando
try:
    #Criação dos dataframes
    s_total = Arquivo("Arquivos CSV/suicidios-total.csv")
    homens_arma = Arquivo("Arquivos CSV/suicidios-homens-armas-fogo.csv")
    mulheres_arma = Arquivo("Arquivos CSV/suicidios-mulheres-armas-fogo.csv")
    s_homens = Arquivo("Arquivos CSV/suicidios-homens.csv")
    s_mulheres = Arquivo("Arquivos CSV/suicidios-mulheres.csv")
    s_regiao = Arquivo("Arquivos CSV/suicidios-região.csv")

    #Inspeção
    if s_total.verifica and homens_arma.verifica and mulheres_arma.verifica and s_homens.verifica and s_mulheres.verifica and s_regiao.verifica:
        print("Arquivos CSV inspecionados com sucesso!")
    else:
        print(f"""
Status de verificação de cada arquivo:
suicidios-total.csv = {s_total.verifica}
suicidios-homens-armas-fogo.csv = {homens_arma.verifica}
suicidios-mulheres-armas-fogo.csv = {mulheres_arma.verifica}
suicidios-homens.csv = {s_homens.verifica}
suicidios-mulheres.csv = {s_mulheres.verifica}
suicidios-região.csv = {s_regiao.verifica}
        """)
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Arquivos CSV inspecionados com sucesso!


In [4]:
#Abrindo conexão com banco local
BancoLocal = BancoDQL('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')
print(dir(BancoLocal)) #Retorna quais propriedades e métodos BancoLocal tem

Conexão bem-sucedida!
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'buscar_dados', 'db_connect', 'encerrar']


In [ ]:
#Criando tabelas no banco local (localhost) PostgresSQL
#BancoLocal.criacao_tabelas()

In [ ]:
#Instanciando DataFrames para usar na inserção dos dados
#colunasPeriodo = BancoLocal.buscar_dados('periodo')[['per_cod', 'ano']]
#colunasRegiao = BancoLocal.buscar_dados('regiao')[['reg_cod', 'nome']]
#regiaoDf = s_regiao.dataframe[['nome', 'período', 'valor']]
#mulherFogoDf = mulheres_arma.dataframe[['período', 'valor']]
#homemFogoDf = homens_arma.dataframe[['período', 'valor']]

In [ ]:
#Realizando inserção dos dados no banco de dados local
#Table periodo:
#BancoLocal.inserir_total(s_total.dataframe)

#Table gen_periodo:
#BancoLocal.inserir_gen(s_homens.dataframe, s_mulheres.dataframe, colunasPeriodo)

#Table reg_periodo:
#BancoLocal.inserir_reg(colunasRegiao, regiaoDf,  colunasPeriodo)

#Table arma_fogo:
#BancoLocal.inserir_dados_fogo(mulherFogoDf, homemFogoDf, colunasPeriodo)

Ideia (meio besta talvez?): Para evitar precisar ficar comentando e descomentando cada subcódigo, que tal um breve menu feito com match case (ou ifs) e input para selecionar, por exemplo, qual inserção quer realizar?

In [ ]:
#Gráfico dos gêneros
dadosGeneros = BancoLocal.buscar_dados('gen_periodo')
dadosSuicidiosGeral = s_total.dataframe['valor'].sum() - dadosGeneros['quantidade'].sum()

soma_generos = dadosGeneros.groupby(['gen_cod','descricao','tipo'], as_index=False).sum()
novo_dado = pd.DataFrame({'gen_cod': [3], 'quantidade': [dadosSuicidiosGeral], 'descricao': ['IGN']})
dados_final = pd.concat([soma_generos, novo_dado], ignore_index=True)

px.pie(values=dados_final['quantidade'] , names=dados_final['descricao'], title="Taxa de suicídio de cada gênero", hole=.4, color_discrete_sequence=px.colors.sequential.Inferno)

In [17]:
dadosArmaFogo = BancoLocal.buscar_dados('arma_fogo')
df = px.data.gapminder()
dadosArmaFogo["size_scaled"] = (dadosArmaFogo["quantidade_total"] - dadosArmaFogo["quantidade_total"].min()) / \
                               (dadosArmaFogo["quantidade_total"].max() - dadosArmaFogo["quantidade_total"].min()) * 50

fig = px.scatter(dadosArmaFogo, x="ano", y="quantidade_total", size="size_scaled", hover_name="ano", log_x=True, size_max=50, color_discrete_sequence=px.colors.sequential.Inferno)
fig.show()

In [31]:
#vertical bar label por genero por periodo aaaa
dadosGeneros = BancoLocal.buscar_dados('gen_periodo')

dadosGeneros = dadosGeneros.rename(columns={'descricao':'Gêneros', 'quantidade':'Quantidade', 'ano':'Anos'})


fig = px.histogram(dadosGeneros, x="Anos", y="Quantidade",
             color='Gêneros', barmode='group',
             height=400)
fig.show()

In [25]:
#quantidade por regiao em cada periodo (linegraph) 
dadosRegPeriodo = BancoLocal.buscar_dados('reg_periodo')

dadosRegPeriodo = dadosRegPeriodo.rename(columns={'quantidade':'Quantidade','ano':'Período','descricao':'Descrição'})

fig = px.line(dadosRegPeriodo, x='Período', y='Quantidade', color='Descrição', markers=True)
fig.show()